In [1]:
from utils import series_ts_extraction_fns

In [2]:
# !pip install ucimlrepo

In [3]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
air_quality = fetch_ucirepo(id=381) 
  
# data (as pandas dataframes) 
X = air_quality.data.features 
y = air_quality.data.targets 

In [4]:
for i in series_ts_extraction_fns:
    i(y)

/home/snape/projects/Benchmarking-Hyper-Parameter-Tuning-Methods-for-Time-Series-Forecasting/utils.py:2322: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  daily_returns = data_close.pct_change().dropna()
/home/snape/projects/Benchmarking-Hyper-Parameter-Tuning-Methods-for-Time-Series-Forecasting/utils.py:2346: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  daily_returns = data_close.pct_change().dropna()
/home/snape/miniconda3/envs/mlenv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3643: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version 